In [ ]:
from selenium import webdriver
from fake_useragent import UserAgent
import time
import pandas as pd
import numpy as np

In [ ]:
def crawling(tap, headless=True):
    taps = ["Summary", "Defensive", "Offensive", "Passing"]
    
    def loading(driver, waiting_time=1, maximum_waiting_time=10):
        print("loading.", end="")
        while True:
            waiting_signal = driver.find_element_by_css_selector("#statistics-table-{}-loading".format(tap.lower())).get_attribute("style")
            time.sleep(waiting_time)
            
            if waiting_signal == "display: none;":
                break
            
            maximum_waiting_time -= 1
            if not maximum_waiting_time:
                break
            
            print(".", end="")
    
    def click_next(driver):
        driver.find_element_by_css_selector("#statistics-paging-{} #next".format(tap.lower())).click()
    
    # Crawling 시작
    driver = webdriver.Chrome()
    driver.get("https://www.whoscored.com/Statistics")
    
    loading(driver)
    
    #해당Tap 클릭
    driver.find_element_by_css_selector("#top-player-stats-options > li:nth-child({}) > a".format(taps.index(tap)+1)).click()
    loading(driver)
    
    #Read Total Pages
    paging_div = driver.find_element_by_css_selector("#statistics-paging-{}".format(tap.lower()))
    total_pages = int(paging_div.text.split('/')[1].split(' ')[0])
    
    #head crawling
    features = []
    theads = driver.find_elements_by_css_selector("#statistics-table-{} > table > thead > tr > th".format(tap.lower()))
    for thead in theads:
        features.append(thead.text)
    
    #body crawling
    players_data = list()
    for page in range(total_pages):
        print("\nCrawling {}page..".format(page + 1))
        tbody_rows = driver.find_elements_by_css_selector("#statistics-table-{} > table > tbody > tr".format(tap.lower()))
        for tbody_row in range(len(tbody_rows)):
            player_data = dict()
            for index_feature in range(len(features)):
                player_data[features[index_feature]] = tbody_rows[tbody_row].find_elements_by_css_selector("td")[index_feature].text
            players_data.append(player_data)
        print("Crawling {}page DONE!\n".format(page + 1))
        click_next(driver)
        loading(driver)
        
    print("\n!!!!ALL CRAWLING SUCCEDED!!!!")
    
    #DataFrame Return
    return pd.DataFrame(columns=features, data=players_data)
    
    

In [ ]:
crawl = crawling("Offensive")
#crawling 변수명에 "Offensive", "Summary", "Defensive", "Passing" 중에 하나 넣으시면 됩니다.
#혹시 돌려보시고 안되시면 말씀해주세요 ㅎㅎ